In [11]:
import requests
import xmltodict
import datetime
import pandas as pd
import glob
import re

In [12]:
# 현재 날짜 확인
now = datetime.datetime.now()
nowDate = now.strftime('%Y%m%d')

nowDate

'20210512'

In [13]:
# 현재 디렉토리에 있는 가장 최신 파일 날짜 확인
ls_all = glob.glob("./dataset/all/*.csv")
if not ls_all:
    oldDate_all = '20200101'
else:
    ls_all.sort()
    oldDate_all = ls_all[-1][22:-8]
    oldDate_csv_all = ls_all[-1][14:]

ls_part = glob.glob("./dataset/part/*.csv")
if not ls_part:
    oldDate_part = '20200101'
else:
    ls_part.sort()
    oldDate_part = ls_part[-1][23:-9]
    oldDate_csv_part = ls_part[-1][15:]

oldDate_all, oldDate_csv_all, oldDate_part, oldDate_csv_part

# all: 전국 통합, part: 시, 도 분할

('20210418',
 'covid19_20210418_all.csv',
 '20210418',
 'covid19_20210418_part.csv')

In [14]:
# 현재 날짜부터 디렉토리에 있는 가장 최신 파일 날짜까지 데이터 다운로드
url_base_all = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
url_base_part = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'
url_serviceKey = 'j8RHAoYLh5NFcdYjbRbdSoOGiuOeHwQZZaU3Mqu78SUYGddoObBnh0Xx3o%2BVPFtZhz5hZaQOf%2B9dMFljJK%2BpLQ%3D%3D'
url_pages = '1000'
url_start_date_all = oldDate_all
url_start_date_part = oldDate_part
url_end_date = nowDate
url_all = url_base_all + '?serviceKey=' + url_serviceKey + '&pageNo=1&numOfRows=' + url_pages + '&startCreateDt=' + url_start_date_all + '&endCreateDt=' + url_end_date
url_part = url_base_part + '?serviceKey=' + url_serviceKey + '&pageNo=1&numOfRows=' + url_pages + '&startCreateDt=' + url_start_date_part + '&endCreateDt=' + url_end_date

In [15]:
req_all = requests.get(url_all).content
req_part = requests.get(url_part).content
xmlObject_all = xmltodict.parse(req_all)
xmlObject_part = xmltodict.parse(req_part)
dict_data_all = xmlObject_all['response']['body']['items']['item']
dict_data_part = xmlObject_part['response']['body']['items']['item']

In [16]:
df_new_all = pd.DataFrame(dict_data_all)
df_new_part = pd.DataFrame(dict_data_part)

In [17]:
for i in range(df_new_part.shape[0]):
    t = re.split(r'년|월|일|시| ', df_new_part['stdDay'][i])
    df_new_part['stdDay'][i] = ''.join(t[0:5:2])

df_new_part

,createDt,deathCnt,defCnt,gubun,gubunCn,gubunEn,incDec,isolClearCnt,isolIngCnt,localOccCnt,overFlowCnt,qurRate,seq,stdDay,updateDt
0,2021-05-12 09:51:43.26,4,3617,검역,隔離區,Lazaretto,13,3434,179,0,13,-,9469,20210512,null
1,2021-05-12 09:51:43.26,1,829,제주,济州,Jeju,13,708,120,13,0,123.59,9468,20210512,null
2,2021-05-12 09:51:43.26,19,4284,경남,庆南,Gyeongsangnam-do,16,3955,310,15,1,127.45,9467,20210512,null
3,2021-05-12 09:51:43.26,79,4359,경북,庆北,Gyeongsangbuk-do,7,3992,288,7,0,163.72,9466,20210512,null
4,2021-05-12 09:51:43.259,11,1195,전남,全南,Jeollanam-do,17,1028,156,17,0,64.08,9465,20210512,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,2021-04-18 09:33:43.425,58,5477,인천,仁川,Incheon,19,5154,265,19,0,185.28,8942,20210418,null
471,2021-04-18 09:33:43.425,219,9152,대구,大邱,Daegu,13,8809,124,13,0,375.62,8941,20210418,null
472,2021-04-18 09:33:43.424,121,4644,부산,釜山,Busan,31,3959,564,30,1,136.12,8940,20210418,null
473,2021-04-18 09:33:43.424,443,35492,서울,首尔,Seoul,205,32377,2672,202,3,364.64,8939,20210418,null


In [18]:
df_new_all = df_new_all.sort_values(by=['stateDt'])
df_new_part = df_new_part.sort_values(by=['stdDay'])
df_new_all = df_new_all.drop(df_new_all.index[0:1])
df_new_part = df_new_part.drop(df_new_part.index[0:20])

In [19]:
df_new_all

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
23,1.3739837378,8414821,8344058,8371,104474,2021-04-19 09:39:21.483,1801,114646,70763,8229412,485,20210419,00:00,null
22,1.3726760312,8458592,8392002,8166,105227,2021-04-20 09:34:02.801,1802,115195,66590,8276807,486,20210420,00:00,null
21,1.3750976675,8497594,8430383,8243,105877,2021-04-21 09:36:56.675,1806,115926,67211,8314457,488,20210421,00:00,null
20,1.3772840512,8537505,8470366,8394,106459,2021-04-22 09:31:58.246,1808,116661,67139,8353705,489,20210422,00:00,null
19,1.3796528397,8583530,8513591,8576,107071,2021-04-23 09:51:40.55,1811,117458,69939,8396133,490,20210423,00:00,null
18,1.3822066912,8632923,8554654,8650,107781,2021-04-24 09:38:23.778,1812,118243,78269,8436411,491,20210424,00:00,null
17,1.3860766953,8654791,8577231,8599,108475,2021-04-25 09:52:51.000,1813,118887,77560,8458344,492,20210425,00:00,null
16,1.3890340008,8672978,8594894,8719,108850,2021-04-26 09:42:14.75,1817,119386,78084,8475508,493,20210426,00:00,2021-04-27 09:37:24.371
15,1.3868895227,8721609,8645101,8383,109695,2021-04-27 09:34:47.417,1820,119898,76508,8525203,494,20210427,00:00,null
14,1.3886694305,8766491,8689325,8597,110248,2021-04-28 09:33:01.863,1821,120666,77166,8568659,495,20210428,00:00,2021-04-30 09:39:12.945


In [20]:
df_new_part
#   속성 설명(all)
#   DECIDE_CNT          확진자수
#   CLEAR_CNT           격리해제 수
#   EXAM_CNT            검사진행 수
#   DEATH_CNT           사망자 수
#   CARE_CNT            치료중 환자 수
#   RESULT_NEG_CNT      결과 음성 수
#   ACC_EXAM_CNT        누적 검사 수
#   ACC_EXAM_COMP_CNT   누적 검사 완료 수
#   ACC_DEF_RATE        누적 확진률
#   CREATE_DT           등록일시분초
#   UPDATE_DT           수정일시분

,createDt,deathCnt,defCnt,gubun,gubunCn,gubunEn,incDec,isolClearCnt,isolIngCnt,localOccCnt,overFlowCnt,qurRate,seq,stdDay,updateDt
438,2021-04-19 09:42:21.775,1,675,제주,济州,Jeju,2,639,35,2,0,100.63,8974,20210419,null
439,2021-04-19 09:42:21.775,17,3440,경남,庆南,Gyeongsangnam-do,35,3069,354,35,0,102.34,8973,20210419,null
440,2021-04-19 09:42:21.774,77,3843,경북,庆北,Gyeongsangbuk-do,19,3515,251,17,2,144.34,8972,20210419,null
441,2021-04-19 09:42:21.774,10,1007,전남,全南,Jeollanam-do,3,909,88,2,1,54.00,8971,20210419,null
442,2021-04-19 09:42:21.774,57,1840,전북,全北,Jeollabuk-do,16,1544,239,16,0,101.25,8970,20210419,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,2021-05-12 09:51:43.257,122,5296,부산,釜山,Busan,10,4923,251,10,0,155.23,9453,20210512,null
17,2021-05-12 09:51:43.257,466,40118,서울,首尔,Seoul,223,36890,2762,220,3,412.16,9452,20210512,null
18,2021-05-12 09:51:43.257,1884,128918,합계,合计,Total,635,119373,7661,613,22,248.65,9451,20210512,null
9,2021-05-12 09:51:43.259,611,36039,경기,京畿,Gyeonggi-do,187,33349,2079,184,3,271.98,9460,20210512,null


In [21]:
df_old_all = pd.read_csv('./dataset/all/covid19_' + oldDate_all + '_all.csv')
df_old_all = df_old_all.drop(['Unnamed: 0'], axis = 1)
df_old_part = pd.read_csv('./dataset/part/covid19_' + oldDate_part + '_part.csv')
df_old_part = df_old_part.drop(['Unnamed: 0'], axis = 1)
df_old_all

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
0,NaN,NaN,NaN,NaN,1,47:33.3,0,0,1,NaN,1,20200101,18:00,21:56.6
1,NaN,NaN,NaN,NaN,1,47:33.3,0,0,1,NaN,1,20200201,18:00,47:33.3
2,NaN,NaN,NaN,NaN,2,22:49.5,2,2,2,NaN,2,20200202,9:00,22:49.5
3,NaN,NaN,NaN,NaN,0,41:17.2,0,15,0,NaN,3,20200203,9:00,41:17.2
4,NaN,NaN,NaN,NaN,0,26:59.6,0,0,0,NaN,4,20200204,0:00,26:59.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,1.363619,8245387.0,8170760.0,7653.0,101983,2021-04-14 09:33:58.653,1782,111418,74627,8059342.0,480,20210414,00:00,2021-04-16 15:24:52.012
440,1.363792,8291125.0,8220904.0,7815.0,102513,2021-04-15 09:38:50.34,1788,112116,70221,8108788.0,481,20210415,00:00,2021-04-16 09:32:37.708
441,1.365080,8333332.0,8262449.0,7937.0,103062,2021-04-16 09:32:10.738,1790,112789,70883,8149660.0,482,20210416,00:00,NaN
442,1.365674,8377778.0,8306739.0,8055.0,103594,2021-04-17 09:40:16.606,1794,113443,71039,8193296.0,483,20210417,00:00,2021-04-18 09:35:59.415


In [22]:
df_all = df_old_all.append(df_new_all)
df_part = df_old_part.append(df_new_part)
df_all.to_csv('./dataset/all/covid19_' + nowDate + '_all.csv', mode='w')
df_part.to_csv('./dataset/part/covid19_' + nowDate + '_part.csv', mode='w')

In [23]:
df_all.info()















<class 'pandas.core.frame.DataFrame'>
Int64Index: 468 entries, 0 to 0
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   accDefRate      437 non-null    object
 1   accExamCnt      437 non-null    object
 2   accExamCompCnt  437 non-null    object
 3   careCnt         436 non-null    object
 4   clearCnt        468 non-null    object
 5   createDt        468 non-null    object
 6   deathCnt        468 non-null    object
 7   decideCnt       468 non-null    object
 8   examCnt         468 non-null    object
 9   resutlNegCnt    437 non-null    object
 10  seq             468 non-null    object
 11  stateDt         468 non-null    object
 12  stateTime       468 non-null    object
 13  updateDt        440 non-null    object
dtypes: object(14)
memory usage: 54.8+ KB
